**文本分类之 TextCNN**

上面一篇文章中，我介绍了一篇利用RNN via Attention解决有毒评论文本分类问题。然而，在工业生产中，RNN,LSTM,GRU等循环神经网络不能并行计算，尽管研究者提出了一些其他的模型，如SRU等。CNN尽管效果在某些时候会稍逊RNN以及RNN的变体，但是CNN在速度方面却有着无可比拟的效果，且由于它的可并行性广被工业界喜爱。

在本文中，我将借助有毒评论分类比赛数据https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge， 介绍TextCNN算法,帮助有需要的朋友入门深度自然语言处理。

关于TextCNN算法，有两篇不错的paper可供参考：

(1) Convolutional Neural Networks for Sentence Classification

(2) A Sensitivity Analysis of (and Practitioners’ Guide to) Convolutional Neural Networks for Sentence Classification


**TextCNN简介**

CNN，做过图像的朋友们都知道，比如图像分类、目标检测、分割等，很多都是利用CNN提取图像的深层次图像表征，并且取得了state-of-the-art performance，在图像领域大放异彩。那么在文本领域是否也可以提取文本的特征呢。2014年Yoon Kim这位大佬揭晓了答案，完全可以，而且也可以取得不错的效果。在文本中，并不是所有的文本都是全部依赖，正如我们在之前一篇文章中利用TFIDF+LR来解决这个问题一样，我们利用ngram信息，捕捉文本的**局部相关性特征**。CNN的原理也是如此，我们可以通过卷积核，来补捉文本的**局部相关性特征**。同时，我们也可以使用多个不同的卷积核，来捕捉多个ngram信息。

**TextCNN原理**

下面我以一张经典的图来简单介绍了以下TextCNN模型。在2014年提出，Yoon Kim使用了卷积 + 最大池化这两个在图像领域非常成功的好基友组合。我们先看一下他的结构。如下图所示，示意图中第一层输入为$7*5$的词向量矩阵，其中词向量维度为5，句子长度为7，然后第二层使用了3组宽度分别为2、3、4的卷积核，图中每种宽度的卷积核使用了两个。其中每个卷积核在整个句子长度上滑动，得到n个激活值，图中卷积核滑动的过程中没有使用padding，因此宽度为4的卷积核在长度为7的句子上滑动得到4个特征值。然后出场的就是卷积的好基友全局池化了，每一个卷积核输出的特征值列向量通过在整个句子长度上取最大值得到了6个特征值组成的feature map来供后级分类器作为分类的依据。

**TextCNN实现**
上面简单介绍了下TextCNN的结构，我们来利用TextCNN来解决我们的有毒评论问题吧。有毒评论问题我们在上几篇文章中已经分析了文本的一些特征，这里我就不下详细描述了，大家有兴趣的化，可以看我之前的几篇文章。本文的主要动机是为了解决RNN的不可并行性，利用CNN的高速并行性。因此，在performance上可以会有些差。话不多说，下面就是我们TextCNN的架构，采用keras简单实现了下，和原始论文的模型有些不一样。

In [ ]:
def TextCNN(maxlen):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    
    conv1 = Conv1D(filters=64, kernel_size=1, padding='same')(x)
    conv1 = MaxPool1D(pool_size=32)(conv1)
    
    
    conv2 = Conv1D(filters=64, kernel_size=2, padding='same')(x)
    conv2 = MaxPool1D(pool_size=32)(conv2)
    
    conv3 = Conv1D(filters=64, kernel_size=3, padding='same')(x)
    conv3 = MaxPool1D(pool_size=32)(conv3)
    
    conv4 = Conv1D(filters=64, kernel_size=4, padding='same')(x)
    conv4 = MaxPool1D(pool_size=32)(conv4)
    
    cnn = concatenate([conv1, conv2, conv3, conv4], axis=-1)
    flat = Flatten()(cnn)

    x = Dense(50, activation="relu")(flat)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model
model=TextCNN(maxlen)
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)
submission = pd.DataFrame.from_dict({'id': test['id']})
y_test = model.predict([X_te], batch_size=1024, verbose=1)
submission[list_classes] = pd.DataFrame(y_test)

上面就是一个非常粗糙的TextCNN模型，我提交了下，效果有点不太理想。如果想要提升下模型效果，可以考虑增加多层卷积，或者残差卷积等等。这仅仅作为入门的一个baseline。
如果你想要运行完整代码可以参考我的github:https://github.com/hecongqing/TextClassification


**完整代码如下**

1、导入后面文章中需要的算法库，本文主要是基于keras实现的(一个高度集成的API文档，后端使用Tenforflow)。

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Conv1D,Concatenate,MaxPool1D,concatenate,Flatten
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import backend as K
from keras.engine.topology import Layer

Using TensorFlow backend.


2、读取我们的数据并替换缺失的值，如果你想要得到很好的效果，可以对这些评论数据进行预处理，比如单词缩写，复数，等等。

In [2]:

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

3、需要对数据集中的评论数据进行向量化。在向量化之前呢，首先需要对这些单词进行编码，也就是将单词转化为数字，因为计算机只能只能识别数字；其实在keras中有个专门的函数，可以帮助我们简单的将单词进行编码。另外，每个句子的长度长短不一，需要选择一个合适的长度，比如100长度的句子可以覆盖90%的文本。在本文中，我们选择句子长度为100，保留文中频率在前20000的单词。

In [3]:

embed_size = 50 #单词向量维度
max_features = 20000 #最多使用多少词，换句话说，也就是我们保留频率前多少词进行embedding，至于之后的我们就忽略掉。
maxlen = 100 #句子选取的长度
#对于以上的三个参数，需要我们对具体的文本进行统计分析。

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train)+list(list_sentences_test))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [4]:
X_t.shape,X_te.shape

((159571, 100), (153164, 100))

4、将单词转化成向量，可以用的方法有很多，word2vec,fasttext,glove等，在本文中，我们用了预训练维度为50的词向量，斯坦福提供的预训练的英文词向量，大家可以在这个链接上下载：http://nlp.stanford.edu/data/glove.6B.zip 。 当然，你也可以自己训练词向量，这里我就不在介绍了。斯坦福也提供了不同的词向量，大家可以去这个官网下载： https://nlp.stanford.edu/projects/glove/ 。

具体地，对于本文来说，我们首先随机初始化一个标准正太矩阵，这样做的有一个好处是，对于有毒评论文本数据的单词在预训练中不存在，我们可以使用一个随机标准正太分布的向量代替它。然后对于存在的单词，我们就使用的预训练的词向量。代码实现的话，大家可以参考下面的，这些代码也可以迁移到其他的任务中。

In [5]:
EMBEDDING_FILE = "../input/glove.6B.50d.txt"
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape      

(20000, 50)

5、TextCNN实现

上面简单介绍了下TextCNN的结构，我们来利用TextCNN来解决我们的有毒评论问题吧。有毒评论问题我们在上几篇文章中已经分析了文本的一些特征，这里我就不下详细描述了，大家有兴趣的化，可以看我之前的几篇文章。本文的主要动机是为了解决RNN的不可并行性，利用CNN的高速并行性。因此，在performance上可以会有些差。话不多说，下面就是我们TextCNN的架构，采用keras简单实现了下，和原始论文的模型有些不一样。

In [ ]:
def TextCNN(maxlen):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    
    conv1 = Conv1D(filters=64, kernel_size=1, padding='same')(x)
    conv1 = MaxPool1D(pool_size=32)(conv1)
    
    
    conv2 = Conv1D(filters=64, kernel_size=2, padding='same')(x)
    conv2 = MaxPool1D(pool_size=32)(conv2)
    
    conv3 = Conv1D(filters=64, kernel_size=3, padding='same')(x)
    conv3 = MaxPool1D(pool_size=32)(conv3)
    
    conv4 = Conv1D(filters=64, kernel_size=4, padding='same')(x)
    conv4 = MaxPool1D(pool_size=32)(conv4)
    
    cnn = concatenate([conv1, conv2, conv3, conv4], axis=-1)
    flat = Flatten()(cnn)

    x = Dense(50, activation="relu")(flat)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model
model=TextCNN(maxlen)

In [12]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 59s 411us/step - loss: 0.0765 - acc: 0.9751 - val_loss: 0.0648 - val_acc: 0.9776
Epoch 2/2
143613/143613 [==============================] - 62s 434us/step - loss: 0.0608 - acc: 0.9785 - val_loss: 0.0630 - val_acc: 0.9782


In [8]:
submission = pd.DataFrame.from_dict({'id': test['id']})
y_test = model.predict([X_te], batch_size=1024, verbose=1)
submission[list_classes] = pd.DataFrame(y_test)

153164/153164 [==============================] - 2s 15us/step


In [10]:
submission.to_csv("textcnn.csv",index=False)

将其结果提交到线上的评测网站，线上提交的结果是分数是0.93xx.

总结
在这篇文章，我主要通过有毒评论分类数据集简单介绍TextCNN模型。TextCNN模型常作于文本分类的baseline

参考

attention机制的实现：https://www.kaggle.com/takuok/bidirectional-lstm-and-attention-lb-0-043

模型参考：https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout

attention论文：Raffel C, Ellis D P W. Feed-forward networks with attention can solve some long-term memory problems[J]. arXiv preprint arXiv:1512.08756, 2015.